## Overview
In the previous mission, we learned how to read JSON into a Spark DataFrame, as well as some basic techniques for interacting with DataFrames. In this mission, we'll learn how to use Spark's SQL interface to query and interact with the data. We'll continue to work with the 2010 U.S. Census data set in this mission. Later on, we'll add other files to demonstrate how to take advantage of SQL to work with multiple data sets.

## Register the DataFrame as a Table
Before we can write and run SQL queries, we need to tell Spark to treat the DataFrame as a SQL table. Spark internally maintains a virtual database within the SQLContext object. This object, which we enter as __sqlCtx__, has methods for registering temporary tables.

To register a DataFrame as a table, call the __registerTempTable()__ [method](https://spark.apache.org/docs/1.5.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.registerTempTable) on that DataFrame object. This method requires one string parameter, name, that we use to set the table __name__ for reference in our SQL queries.

#### Instructions
* Use the __registerTempTable()__ [method](https://spark.apache.org/docs/1.5.0/api/python/pyspark.sql.html#pyspark.sql.DataFrame.registerTempTable) method to register the DataFrame __df__ as a table named __census2010__.
* Then, run the SQLContext method __tableNames__ to return the list of tables.
* Assign the resulting list to __tables__, and use the __print__ function to display it.

In [1]:
# Find path to PySpark
import findspark
findspark.init()

# Import PySpark & initalize SparkContext object
import pyspark
sc = pyspark.SparkContext()

# Import SQLContext
from pyspark.sql import SQLContext

# Pass in the SparkContext object `sc`
sqlCtx = SQLContext(sc)